In [33]:
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
import pandas as pd
from tqdm import tqdm
import json
import time

# custom header for requests add javascript header
headers = {
  'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3',
}

# get the data from the website
url1 = 'https://api.jumpit.co.kr/api/positions?sort=rsp_rate&highlight=false&page={}' # 점핏
url2 = 'https://www.jobplanet.co.kr/api/v3/job/postings?order_by=aggressive&occupation_level1=11600,11912&page={}&page_size=8' # 잡플래닛
url3 = 'https://career.programmers.co.kr/api/job_positions?page={}' # 프로그래머스
url4 = 'https://www.rocketpunch.com/api/jobs/template?page={}&job=1' # 로켓펀치

In [6]:
# 점핏 crawler
def jumpit_crawler():
  data_list = []

  res = requests.get(url1.format(1), headers=headers)
  res = json.loads(res.text)

  data_list += res['result']['positions'] # add to the list

  # iterate through the pages totalCount / 16(per page)
  for i in range(2, res['result']['totalCount'] // 16 + 1):
    res = requests.get(url1.format(i), headers=headers)
    res = json.loads(res.text)

    data_list += res['result']['positions'] # add to the list
    time.sleep(2) # for sake of politeness

  return data_list

jumpit_list = jumpit_crawler()

In [40]:
# print(jumpit_list)
# print(len(jumpit_list))
# df_jp = pd.DataFrame(jumpit_list)
# df_jp.to_csv('jumpit_list.csv', index=False)

# read from csv
df_jp = pd.read_csv('jumpit_list.csv')

display(df_jp.head(2))
display(df_jp.shape)

,id,jobCategory,logo,imagePath,title,companyName,techStacks,scrapCount,viewCount,newcomer,minCareer,maxCareer,locations,alwaysOpen,closedAt,companyProfileId,celebration,scraped,applied
0,21310,"서버/백엔드 개발자,웹 풀스택 개발자",https://cdn.jumpit.co.kr/lg/images/hmlee_4/202...,https://cdn.jumpit.co.kr/lg/images/hmlee_4/202...,개발 팀장,한국해양기상기술,"['Java', 'Spring', 'PostgreSQL', 'Spring Boot'...",135,1752,False,10,30,['서울 구로구'],True,9999-12-31T23:59:59,5506,70,False,False
1,22041,"웹 풀스택 개발자,개발 PM",https://cdn.jumpit.co.kr/lg/images/139324/2023...,https://cdn.jumpit.co.kr/lg/images/139324/2023...,[5년 이상 리더급] 풀스택 소프트웨어 엔지니어,다이얼로그디자인에이전시,"['Django', 'Python', 'Flask', 'React', 'Flutte...",2,590,False,5,7,['서울 송파구'],False,2024-04-30T23:59:59,5135,70,False,False


(2160, 19)

In [41]:
jp_detail_url = 'https://api.jumpit.co.kr/api/position/{}' # 점핏 상세페이지 url

jumpit_serviceInfo_list = []     # 기업/서비스 소개
jumpit_responsibility_list = []     # 주요업무
jumpit_qualifications_list = []     # 자격요건
jumpit_preferredRequirements_list = []     # 우대사항
jumpit_welfares_list = []     # 복지 및 혜택

for i in tqdm(df_jp.iterrows()):
  res = requests.get(jp_detail_url.format(i[1]['id']), headers=headers)
  jumpit_result = json.loads(res.text)['result']
  
  jumpit_serviceInfo_list += [jumpit_result['serviceInfo']]
  jumpit_responsibility_list += [jumpit_result['responsibility']]
  jumpit_qualifications_list += [jumpit_result['qualifications']]
  jumpit_preferredRequirements_list += [jumpit_result['preferredRequirements']]
  jumpit_welfares_list += [jumpit_result['welfares']]
  time.sleep(0.1) # for sake of politeness

detail_columns = {'serviceInfo' : jumpit_serviceInfo_list, \
                'responsibility' : jumpit_responsibility_list, \
                'qualifications' : jumpit_qualifications_list, \
                'preferredRequirements' : jumpit_preferredRequirements_list, \
                'welfares' : jumpit_welfares_list}

df_jp_detail = pd.DataFrame(detail_columns)

2160it [06:45,  5.33it/s]


In [42]:
df_jp_full = pd.concat([df_jp, df_jp_detail], axis = 1)

df_jp_full.to_pickle('jumpit_full.pkl')

In [11]:
# 잡플래닛 crawler
def jobplanet_crawler():
  # selenium webdriver with headless option
  options = webdriver.ChromeOptions()
  options.add_argument('headless')
  driver = webdriver.Chrome(options=options)

  # add option for evade cloudflare bot detection
  driver.execute_cdp_cmd('Network.setUserAgentOverride', {
    'userAgent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
  })

  data_list = []

  # get data from the url2
  driver.get(url2.format(1))

  # get the data from the website
  res = driver.page_source
  soup = BeautifulSoup(res, 'html.parser')
  res = json.loads(soup.text)

  data_list += res['data']['recruits'] # add to the list

  for i in range(2, res['data']['total_count'] // len(res['data']['recruits']) + 1):
    driver.get(url2.format(i))
    driver.implicitly_wait(10) # wait for the page to load

    res = driver.page_source
    soup = BeautifulSoup(res, 'html.parser')
    res = json.loads(soup.text)

    data_list += res['data']['recruits']
    time.sleep(2) # for sake of politeness

  driver.quit() # close the driver

  return data_list

jobplanet_list = jobplanet_crawler()

In [13]:
print(len(jobplanet_list))
df_jp = pd.DataFrame(jobplanet_list)
df_jp.to_csv('jobplanet_list.csv', index=False)

# df_jp = pd.read_csv('jobplanet_list.csv')

display(df_jp.head(2))

1432


,id,bookmark,image,title,occupation_names,is_fit_messages,fee,job_applicant_type,annual,recruitment_text,...,posting_type,posting_apply_type,skills,reward,reward_text,reward_type_text,deadline_message,company,created_at,updated_at
0,1273751,False,https://jpassets.jobplanet.co.kr/production/up...,LabVIEW 프로그램 개발자 (신입),"{'level1': ['개발'], 'level2': ['소프트웨어 개발']}",False,7%,user,"{'type': [1], 'years': 0, 'maximum_years': 0, ...",[신입],...,recruitment,jobplanet,"[C#, python, LabVIEW, 응용프로그램개발, 어플리케이션 개발, c/c++]",1000000,100만원,취업 축하금,상시채용,"{'id': 138066, 'logo': 'https://jpassets.jobpl...",2024-04-16T19:16:33.000+09:00,2024-04-17T14:09:11.000+09:00
1,1273749,False,https://jpassets.jobplanet.co.kr/production/up...,서버 네트워크 엔지니어,"{'level1': ['개발'], 'level2': ['기술지원', '네트워크/보안...",False,7%,user,"{'type': [2], 'years': 3, 'maximum_years': 5, ...",[3 ~ 5년],...,recruitment,jobplanet,"[서버개발, 유지보수, 네트워크보안, tcpip, 기술지원, 전문연구요원, 자율주행...",1000000,100만원,취업 축하금,상시채용,"{'id': 138066, 'logo': 'https://jpassets.jobpl...",2024-04-16T19:12:11.000+09:00,2024-04-17T14:08:48.000+09:00


In [18]:
jp_detail_url = 'https://www.jobplanet.co.kr/api/v1/job/postings/{}' # 잡플래닛 상세페이지 url

jp_detail_list = []
for i in tqdm(df_jp.iterrows()):
  # selenium webdriver with headless option
  options = webdriver.ChromeOptions()
  options.add_argument('headless')
  driver = webdriver.Chrome(options=options)

  # add option for evade cloudflare bot detection
  driver.execute_cdp_cmd('Network.setUserAgentOverride', {
    'userAgent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
  })

  # get data from the detail page
  driver.get(jp_detail_url.format(i[1]['id']))
  driver.implicitly_wait(10) # wait for the page to load

  res = driver.page_source
  soup = BeautifulSoup(res, 'html.parser')
  res = json.loads(soup.text)

  jp_detail_list += [res['data']]
  time.sleep(0.1) # for sake of politeness

0it [00:00, ?it/s]

1432it [20:38,  1.16it/s]


In [21]:
display(len(jp_detail_list))
display(jp_detail_list[0])
display(jp_detail_list[1])

df_jp_detail = pd.DataFrame(jp_detail_list)
df_jp_full = pd.concat([df_jp, df_jp_detail], axis = 1)

df_jp_full.to_pickle('jobplanet_full.pkl')

1432

{'open': True,
 'title': 'LabVIEW 프로그램 개발자 (신입)',
 'apply_type': None,
 'applied': False,
 'bookmark': False,
 'is_fit_messages': False,
 'name': '(주)에이스웍스코리아',
 'company_id': 138066,
 'score': '3.0',
 'working_area': ['서울'],
 'images': [],
 'appeal': None,
 'simple_comment': None,
 'matching_keyword': None,
 'keywords': [],
 'end_at': '2024.07.16',
 'immediate_opening': True,
 'recruitment_deadline': False,
 'occupations': ['소프트웨어 개발'],
 'occupations_level1': ['개발'],
 'fee': '7%',
 'job_applicant_type': 'user',
 'posting_from': 'partners',
 'annual': {'types': [1],
  'year': 0,
  'maximum_year': 0,
  'text': '신입',
  'preferential': False},
 'recruitment_text': ['신입'],
 'job_type': '정규직',
 'salary': ' (  ~  )',
 'skills': ['C#', 'python', 'LabVIEW', '응용프로그램개발', '어플리케이션 개발', 'c/c++'],
 'introduction': '미래모빌리티 솔루션 기업 ACEWORKS!\r\n\r\n에이스웍스 코리아는 EV 검증장비 국내 1위 기업 CONTROLWORKS 와 자율주행 전문 스타트업 ACELAB이 합병하여 탄생한 미래모빌리티 전문 솔루션 기업입니다. 전직원 중 연구인력이 80% 이상인 연구개발 중심 기업입니다.\r\n\r\n다년간 축적된 임베디드 하드웨어와 소

{'open': True,
 'title': '서버 네트워크 엔지니어',
 'apply_type': None,
 'applied': False,
 'bookmark': False,
 'is_fit_messages': False,
 'name': '(주)에이스웍스코리아',
 'company_id': 138066,
 'score': '3.0',
 'working_area': ['서울'],
 'images': [],
 'appeal': None,
 'simple_comment': None,
 'matching_keyword': None,
 'keywords': [],
 'end_at': '2024.07.16',
 'immediate_opening': True,
 'recruitment_deadline': False,
 'occupations': ['기술지원', '네트워크/보안/운영'],
 'occupations_level1': ['개발'],
 'fee': '7%',
 'job_applicant_type': 'user',
 'posting_from': 'partners',
 'annual': {'types': [2],
  'year': 3,
  'maximum_year': 5,
  'text': '경력',
  'preferential': False},
 'recruitment_text': ['3 ~ 5년'],
 'job_type': '정규직',
 'salary': ' (  ~  )',
 'skills': ['서버개발',
  '유지보수',
  '네트워크보안',
  'tcpip',
  '기술지원',
  '전문연구요원',
  '자율주행',
  'c/c++'],
 'introduction': '미래모빌리티 솔루션 기업 ACEWORKS!\r\n\r\n에이스웍스 코리아는 EV 검증장비 국내 1위 기업 CONTROLWORKS 와 자율주행 전문 스타트업 ACELAB이 합병하여 탄생한 미래모빌리티 전문 솔루션 기업입니다. 전직원 중 연구인력이 80% 이상인 연구개발 중심 기업입니다.

In [3]:
# 프로그래머스 crawler
def programmers_crawler():
  data_list = []
  res = requests.get(url3.format(1), headers=headers)
  res = json.loads(res.text)

  data_list += res['jobPositions']

  for i in range(2, res['totalPages'] + 1):
    res = requests.get(url3.format(i), headers=headers)
    res = json.loads(res.text)

    data_list += res['jobPositions']
    time.sleep(1)

  return data_list

programmers_list = programmers_crawler()

In [22]:
print(len(programmers_list))
df_pg = pd.DataFrame(programmers_list)
# delete row where status is 'hidden'
df_pg = df_pg[df_pg['status'] != 'hidden']
display(df_pg.shape)

df_pg.to_csv('programmers_list.csv', index=False)

# df_pg = pd.read_csv('programmers_list.csv')

display(df_pg.head(2))

1207


(1202, 23)

,id,address,career,careerRange,companyId,jobType,maxSalary,minSalary,personalized,signingBonus,...,createdAt,updatedAt,careerOption,jobCategoryIds,period,technicalTags,teamTechnicalTags,url,isSynced,company
0,25558,"서울특별시 성동구 성수이로10길 14, 904호, 905호, 10층(성수동2가, 에...",3 ~ 10년,3...10,7147,full_time,NaN,NaN,None,500000,...,2024-01-10T10:57:22.636+09:00,2024-04-17T16:08:01.985+09:00,True,[4],상시 채용,"[{'id': 1234, 'name': 'ReactJS', 'taggings_cou...","[C#, Java, Spring, REST API, TCP/IP, ReactJS, ...",/job_positions/25558,False,"{'id': 7147, 'name': '쿠프마케팅', 'address': '서울특별..."
1,27200,"서울 서초구 바우뫼로 128, (양재동), 비스텔타워",3 ~ 15년,3...15,6885,full_time,NaN,NaN,None,500000,...,2024-04-17T15:55:37.543+09:00,2024-04-17T15:57:09.981+09:00,True,"[1, 9]",상시 채용,"[{'id': 1182, 'name': 'Java', 'taggings_count'...",[],/job_positions/27200,False,"{'id': 6885, 'name': '비스텔리젼스', 'address': '서울 ..."


In [23]:
pg_detail_url = 'https://career.programmers.co.kr/api/job_positions/{}' # 프로그래머스 상세페이지 api_url

pg_detail_list = []

for i in tqdm(df_pg.iterrows()):
  res = requests.get(pg_detail_url.format(i[1]['id']), headers=headers)
  res = json.loads(res.text)
  try:
    res = res['jobPosition']

    # push res['technicalTags'] items ['name'] value to res['tech_stack']
    res['tech_stack'] = [tag['name'] for tag in res['technicalTags']]
    res['team_stack'] = [tag['name'] for tag in res['teamTechnicalTags']]
  except:
    # missing id will cause error (when the job posting is modified id will be changed)
    print(i)

  pg_detail_list += [res]
  time.sleep(0.1) # for sake of politeness

162it [00:54,  3.17it/s]

(161, id                                                               14775
address                                                서울 강남구 테헤란로 410
career                                                           경력 무관
careerRange                                                       None
companyId                                                         8755
jobType                                                       contract
maxSalary                                                          NaN
minSalary                                                          NaN
personalized                                                      None
signingBonus                                                    500000
status                                                        released
title                                  [네이버클라우드&웅진씽크빅-(주)배컴] 프론트앤드 매니저
startAt                                  2024-04-03T11:00:00.000+09:00
createdAt                                2022-10-20T14:29:14.825+09:00


1202it [05:47,  3.46it/s]


In [25]:
print(len(pg_detail_list))
display(pg_detail_list[0])
display(pg_detail_list[1])

df_pg_detail = pd.DataFrame(pg_detail_list)
df_pg_full = pd.concat([df_pg, df_pg_detail], axis = 1)

df_pg_full.to_pickle('programmers_full.pkl')

1202


{'id': 25558,
 'address': '서울특별시 성동구 성수이로10길 14, 904호, 905호, 10층(성수동2가, 에이스하이엔드성수타워) ',
 'career': '3 ~ 10년',
 'careerRange': '3...10',
 'companyId': 7147,
 'jobType': 'full_time',
 'maxSalary': None,
 'minSalary': None,
 'personalized': None,
 'signingBonus': 500000,
 'status': 'released',
 'title': '[쿠프마케팅그룹] 페이즈 북앤라이프 프론트엔드 개발자',
 'startAt': '2024-04-17T10:57:00.000+09:00',
 'createdAt': '2024-01-10T10:57:22.636+09:00',
 'updatedAt': '2024-04-17T16:08:01.985+09:00',
 'careerOption': True,
 'jobCategoryIds': [4],
 'period': '상시 채용',
 'technicalTags': [{'id': 1234,
   'name': 'ReactJS',
   'taggings_count': 104033,
   'category': 'technical',
   'approved': True,
   'display_order': 1000.0},
  {'id': 1257,
   'name': 'TypeScript',
   'taggings_count': 62872,
   'category': 'technical',
   'approved': True,
   'display_order': 1000.0},
  {'id': 1575,
   'name': 'Next.js',
   'taggings_count': 26665,
   'category': 'technical',
   'approved': True,
   'display_order': 1000.0},
  {'id': 

{'id': 27200,
 'address': '서울 서초구 바우뫼로 128, (양재동), 비스텔타워',
 'career': '3 ~ 15년',
 'careerRange': '3...15',
 'companyId': 6885,
 'jobType': 'full_time',
 'maxSalary': None,
 'minSalary': None,
 'personalized': None,
 'signingBonus': 500000,
 'status': 'released',
 'title': 'Application Engineer',
 'startAt': '2024-04-17T15:52:00.000+09:00',
 'createdAt': '2024-04-17T15:55:37.543+09:00',
 'updatedAt': '2024-04-17T15:57:09.981+09:00',
 'careerOption': True,
 'jobCategoryIds': [1, 9],
 'period': '상시 채용',
 'technicalTags': [{'id': 1182,
   'name': 'Java',
   'taggings_count': 138995,
   'category': 'technical',
   'approved': True,
   'display_order': 1000.0},
  {'id': 1230,
   'name': 'Python',
   'taggings_count': 103451,
   'category': 'technical',
   'approved': True,
   'display_order': 1000.0},
  {'id': 1191,
   'name': 'Apache Kafka',
   'taggings_count': 7101,
   'category': 'technical',
   'approved': True,
   'display_order': 1000.0},
  {'id': 1196,
   'name': 'Linux',
   'tagging

In [44]:
# 로켓펀치 crawler
def rocketpunch_crawler():
  session = requests.Session()
  res = session.get(url4.format(1), headers=headers)
  res = json.loads(res.text)
  soup = BeautifulSoup(res['data']['template'], 'html.parser')

  page_size = soup.find('div', {'class': 'tablet computer large screen widescreen only'}).find_all('a', {'class': 'item'})[-1].text.strip()

  data_list = parse_page(soup)

  for i in range(2, int(page_size) + 1):
    res = session.get(url4.format(i), headers=headers)
    res = json.loads(res.text)
    soup = BeautifulSoup(res['data']['template'], 'html.parser')
    data_list.extend(parse_page(soup))
    time.sleep(2) # for sake of politeness

  return data_list

def parse_page(soup):
  data_list = []
  for company in soup.find_all('div', {'class': 'company item'}):
    company_data = {}
    company_data['company_id'] = company['data-company_id']
    for content in company.find_all('div', {'class': 'content'}):
      company_data['company_name'] = content.find('a', {'class': 'company-name nowrap header name'}).text.strip()
      company_data['description'] = content.find('div', {'class': 'description'}).text.strip()
      company_data['job_list'] = []
      for job_detail in content.find_all('div', {'class': 'job-detail'}):
        job_data = {}
        job_data['job_id'] = job_detail.find('a', {'class': 'nowrap job-title'})['href'].split('/')[2]
        job_data['job_title'] = job_detail.find('a', {'class': 'nowrap job-title'}).text.strip()
        job_data['job_info'] = job_detail.find('div', {'class': 'job-stat-info'}).text.strip().split(' / ')
        job_data['job_date'] = job_detail.find('div', {'class': 'job-dates'}).text.strip()
        company_data['job_list'].append(job_data)
      data_list.append(company_data)
  return data_list


rocketpunch_list = rocketpunch_crawler()

In [64]:
display(len(rocketpunch_list))
df_rp = pd.DataFrame(rocketpunch_list)
# unpack job_list to multiple rows
df_rp = df_rp.explode('job_list')
df_rp = pd.concat([df_rp.drop(['job_list'], axis=1), df_rp['job_list'].apply(pd.Series)], axis=1).reset_index(drop=True)
df_rp.to_csv('rocketpunch_list.csv', index=False)

# df_rp = pd.read_csv('rocketpunch_list.csv')

display(df_rp.shape)
display(df_rp.head(2))

141

(281, 7)

,company_id,company_name,description,job_id,job_title,job_info,job_date
0,85579,주식회사 노타,인공지능으로 세상을 편리하게 하자는 철학을 실천하는 기업입니다 :),150261,AI Research Engineer Intern (EdgeFM팀),[인턴],04/30 마감
1,85579,주식회사 노타,인공지능으로 세상을 편리하게 하자는 철학을 실천하는 기업입니다 :),150259,AI Research Engineer (EdgeFM팀),[경력],05/04 마감


In [69]:
rp_detail_url = 'https://www.rocketpunch.com/jobs/{}' # 로켓펀치 상세페이지 url

rp_detail_list = []

for i in tqdm(df_rp.iterrows()):
  res = requests.get(rp_detail_url.format(i[1]['job_id']), headers=headers)
  soup = BeautifulSoup(res.text, 'html.parser')


  # get data <script type="application/ld+json">
  script = soup.find('script', {'type': 'application/ld+json'}).text
  data = json.loads(script)
  # get data <div class="job-specialties">
  specialties = soup.find('div', {'class': 'job-specialties'}).find_all('a')
  specialties = [specialty.text.strip() for specialty in specialties]
  # get data <div class="content break">
  requirement = soup.find('div', {'class': 'content break'}).text.strip()

  data['requirement'] = requirement
  data['specialties'] = specialties

  rp_detail_list.append(data)
  time.sleep(0.1) # for sake of politeness

print(len(rp_detail_list))

0it [00:00, ?it/s]

{'@context': 'http://schema.org',
 '@type': 'JobPosting',
 'datePosted': '2024-04-04',
 'description': '• On-device foundation model을 위해 파이토치 모델을 최적화합니다.<br />- TensorRT 등 모델 변환을 통해 추론 속도를 개선합니다.<br />- Quantization, knowledge distillation, pruning 등 기법을 적용하여 효율적인 모델을 만듭니다.<br />• LLM, LMM, text-to-image 등 Gen AI 모델에 대한 최적화를 진행합니다.<br />- TensorRT, vLLM 등 더 빠른 SDK나 라이브러리로 모델을 변환하여 추론 속도를 개선합니다.<br />- KVCache (LLM), step distillation (text-to-image) 등 특정 task에서 사용되는 최적화 기법들을 노타의 모델에 적용합니다.<br />• Foundation model의 효율적 설계 및 사용에 대해 기법을 연구하고 제품화를 진행합니다.<br />- 효율적인 foundation model을 만들고, 이를 디바이스에 탑재하여 노타가 강점을 가지는 분야(ex. ITS)에서 실제로 사용할 수 있는 제품을 만듭니다.<br /><br /><br />[Edge FM 팀을 소개합니다.]<br />Edge Foundation Models (EdgeFM) 팀은 기술을 경량화하고 최적화하여 Edge device에서 Foundation Model을 실행할 수 있도록 연구하고 있습니다. 최신 경량화 기술과 최적화 기법을 활용하여, 중앙 서버 없이도 사람 수준의 지식과 추론 능력을 갖춘 AI 모델을 사용할 수 있게 하며, 언제 어디서든 Edge 장치만으로도 AI 기술을 쉽게 활용할 수 있는 시대를 준비하며, 노타의 선도적인 지식과 기술을 정립하고 있는 조직입니다.<br /><br />해당 포지션으로 합류하신다면 Foundation Mode

0it [00:00, ?it/s]

0


In [65]:
print(len(rp_detail_list))
print(rp_detail_list[0])
print(rp_detail_list[1])

df_rp_detail = pd.DataFrame(rp_detail_list)
df_rp_full = pd.concat([df_rp, df_rp_detail], axis = 1)

df_rp_full.to_pickle('rocketpunch_full.pkl')

281
{'@context': 'http://schema.org', '@type': 'JobPosting', 'datePosted': '2024-04-04', 'description': '• On-device foundation model을 위해 파이토치 모델을 최적화합니다.<br />- TensorRT 등 모델 변환을 통해 추론 속도를 개선합니다.<br />- Quantization, knowledge distillation, pruning 등 기법을 적용하여 효율적인 모델을 만듭니다.<br />• LLM, LMM, text-to-image 등 Gen AI 모델에 대한 최적화를 진행합니다.<br />- TensorRT, vLLM 등 더 빠른 SDK나 라이브러리로 모델을 변환하여 추론 속도를 개선합니다.<br />- KVCache (LLM), step distillation (text-to-image) 등 특정 task에서 사용되는 최적화 기법들을 노타의 모델에 적용합니다.<br />• Foundation model의 효율적 설계 및 사용에 대해 기법을 연구하고 제품화를 진행합니다.<br />- 효율적인 foundation model을 만들고, 이를 디바이스에 탑재하여 노타가 강점을 가지는 분야(ex. ITS)에서 실제로 사용할 수 있는 제품을 만듭니다.<br /><br /><br />[Edge FM 팀을 소개합니다.]<br />Edge Foundation Models (EdgeFM) 팀은 기술을 경량화하고 최적화하여 Edge device에서 Foundation Model을 실행할 수 있도록 연구하고 있습니다. 최신 경량화 기술과 최적화 기법을 활용하여, 중앙 서버 없이도 사람 수준의 지식과 추론 능력을 갖춘 AI 모델을 사용할 수 있게 하며, 언제 어디서든 Edge 장치만으로도 AI 기술을 쉽게 활용할 수 있는 시대를 준비하며, 노타의 선도적인 지식과 기술을 정립하고 있는 조직입니다.<br /><br />해당 포지션으로 합류하신다면 Foundation Mod